In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
from scipy import stats
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
import joblib


In [2]:
# Read the CSV file into a Pandas DataFrame
df = pd.read_csv('Data/final_data.csv')
df.dropna(subset=df.columns, inplace=True)
# Review the DataFrame
df

,Rnd,Pick,Player,Pos,AP1,PB,College/Univ,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Drafted_Binary,Pos_encoded,College/Univ_encoded
0,1.0,1.0,Andrew Luck,QB,0.0,1.0,Stanford,76.0,234.0,4.590000,36.000000,15.000000,124.000000,6.800000,4.280000,1.0,1,0
1,1.0,3.0,Trent Richardson,RB,0.0,0.0,Alabama,69.0,228.0,4.480000,34.626404,18.960000,120.636905,7.038725,4.262870,1.0,2,1
2,1.0,4.0,Matt Kalil,T,0.0,1.0,USC,79.0,306.0,4.960000,27.000000,30.000000,105.500000,7.330000,4.650000,1.0,3,2
3,1.0,5.0,Justin Blackmon,WR,0.0,0.0,Oklahoma St.,73.0,207.0,4.460000,35.839015,14.000000,122.860465,6.944684,4.222308,1.0,4,3
4,1.0,6.0,Morris Claiborne,DB,0.0,0.0,LSU,71.0,188.0,4.500000,34.500000,15.413534,118.000000,7.010000,4.120000,1.0,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2274,7.0,251.0,Isiah Pacheco,RB,0.0,0.0,Rutgers,70.0,216.0,4.370000,33.000000,18.960000,118.000000,7.038725,4.262870,1.0,2,47
2275,7.0,255.0,Trenton Gill,P,0.0,0.0,North Carolina St.,76.0,220.0,4.795385,31.800000,23.000000,117.200000,7.030000,4.130000,1.0,12,41
2276,7.0,256.0,Jesse Luketa,LB,0.0,0.0,Penn St.,75.0,253.0,4.890000,37.500000,22.419847,114.000000,7.079314,4.281776,1.0,6,29
2277,7.0,257.0,Marquis Hayes,OL,0.0,0.0,Oklahoma,77.0,318.0,5.300000,23.500000,27.140000,102.000000,7.720667,4.706000,1.0,13,30


In [3]:
# df['Pro_Bowl'] = df['Pro_Bowl'].replace([2, 3, 4, 5, 6], 1)
# df.head()

In [4]:
# Define the Z-score threshold for outlier removal
z_score_threshold = 3

# Calculate Z-scores for the 'Numerical_Column'
z_scores = stats.zscore(df['40yd'])

# Create a mask to identify outliers
outlier_mask = z_scores < z_score_threshold

# Apply the mask to keep only non-outliers in DataFrame df
df = df[outlier_mask]

In [5]:
columns_to_drop = ['Drafted_Binary', 'Player', 'Pos', 'College/Univ', 'AP1', 'AP1']
# df.reset_index(drop=True, inplace=True)
X= df.drop(columns=columns_to_drop, axis=1)  # Features, excluding the label column
X

,Rnd,Pick,PB,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Pos_encoded,College/Univ_encoded
0,1.0,1.0,1.0,76.0,234.0,4.590000,36.000000,15.000000,124.000000,6.800000,4.280000,1,0
1,1.0,3.0,0.0,69.0,228.0,4.480000,34.626404,18.960000,120.636905,7.038725,4.262870,2,1
2,1.0,4.0,1.0,79.0,306.0,4.960000,27.000000,30.000000,105.500000,7.330000,4.650000,3,2
3,1.0,5.0,0.0,73.0,207.0,4.460000,35.839015,14.000000,122.860465,6.944684,4.222308,4,3
4,1.0,6.0,0.0,71.0,188.0,4.500000,34.500000,15.413534,118.000000,7.010000,4.120000,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2274,7.0,251.0,0.0,70.0,216.0,4.370000,33.000000,18.960000,118.000000,7.038725,4.262870,2,47
2275,7.0,255.0,0.0,76.0,220.0,4.795385,31.800000,23.000000,117.200000,7.030000,4.130000,12,41
2276,7.0,256.0,0.0,75.0,253.0,4.890000,37.500000,22.419847,114.000000,7.079314,4.281776,6,29
2277,7.0,257.0,0.0,77.0,318.0,5.300000,23.500000,27.140000,102.000000,7.720667,4.706000,13,30


In [6]:
y = df['Drafted_Binary']
# X = df.drop(columns='Pro_Bowl')
y

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
2274    1.0
2275    1.0
2276    1.0
2277    1.0
2278    1.0
Name: Drafted_Binary, Length: 2278, dtype: float64

In [7]:
# X = pd.get_dummies(X)

In [8]:
# Create an Isolation Forest for anomaly detection
iso_forest = IsolationForest(contamination=0.01, random_state=20)

# Fit the Isolation Forest to the data
iso_forest.fit(X)

# Predict anomalies (-1) and inliers (1)
anomaly_predictions = iso_forest.predict(X)

# Create a mask to identify anomalies
anomaly_mask = anomaly_predictions == -1

# Filter out anomalies from the dataset
X_clean = X[~anomaly_mask]
y_clean = y[~anomaly_mask]

In [9]:
# Assuming you have X (features) and y (target) prepared
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=.7, random_state=20)


In [10]:
# Create an One-Class SVM model for anomaly detection
ocsvm_model = OneClassSVM(nu=0.05)  # Adjust 'nu' as needed

# Fit the OC-SVM model on your clean training data
ocsvm_model.fit(X_train)

# Predict anomalies (-1) and inliers (1) for the entire dataset
anomaly_predictions = ocsvm_model.predict(X)

# Create a mask to identify anomalies
anomaly_mask = anomaly_predictions == -1

# Filter out anomalies from the dataset
X_clean = X[~anomaly_mask]
y_clean = y[~anomaly_mask]


In [11]:

# Apply SMOTE to balance the class distribution
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_clean, y_clean)

# Split the resampled data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.5, random_state=42)

In [12]:
# Instantiate the Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=5000, max_depth=20, random_state=20, class_weight='balanced')

In [13]:
# Train the model on the training data
rf_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=20, n_estimators=5000,
                       random_state=20)

In [14]:
# Make predictions on the test data
y_pred = rf_model.predict(X_test)

In [15]:
# Get predicted probabilities for the positive class ('Pro_Bowl')
y_probabilities = rf_model.predict_proba(X_test)[:, 1]  # Probability of 'Pro_Bowl'

# Adjust the threshold (e.g., set it to 0.15 for higher precision)
custom_threshold = 0.5
y_custom_pred = (y_probabilities > custom_threshold).astype(int)  # Convert to 0 or 1

# Evaluate the model's performance with the custom threshold
accuracy = accuracy_score(y_test, y_custom_pred)
precision = precision_score(y_test, y_custom_pred)
recall = recall_score(y_test, y_custom_pred)
f1 = f1_score(y_test, y_custom_pred)

In [16]:
# Evaluate the model's accuracy

print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)



Accuracy: 0.9804469273743017


In [17]:
# Generate a classification report
report = classification_report(y_test, y_custom_pred)
print(report)

              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98      1064
         1.0       0.99      0.97      0.98      1084

    accuracy                           0.98      2148
   macro avg       0.98      0.98      0.98      2148
weighted avg       0.98      0.98      0.98      2148



In [20]:
probowl_prediction_model_file = 'josh_model.joblib'
joblib.dump(rf_model, probowl_prediction_model_file)

['josh_model.joblib']